In [51]:
import zipfile
import os
import glob
import io
import re
import random

In [49]:
akuta_dir = "akutagawa"
miya_dir = "miyazawa"
akuta_tsv = "aozora-akutagawa.tsv"
miya_tsv = "aozora-miyazawa.tsv"

In [54]:
# テキストの下処理
def preprocess(file):
    kruby = re.compile('｜([^《]+)《[^》]+》')
    ruby = re.compile('《[^》]+》')
    annotate = re.compile('［＃[^］]+］')
    nonpr = re.compile('[\x00-\x09\x0b-\x1f]+')
    # ヘッダの削除
    ## ----- が2回出現するまでスキップ
    for line in file:
        if line.startswith('--------'):
            break
    for line in file:
        if line.startswith('--------'):
            break
    # 本文の処理
    for line in file:
        if line.startswith('底本：'): # 本文終わり
            break
        if line[:-1].strip() == "":
            continue
        # 本文として処理
        line = kruby.sub('\1', line)
        line = ruby.sub('', line)
        line = annotate.sub('', line)
        line = nonpr.sub('', line)
        yield line

In [57]:
# tsvへの変換
def to_tsv(line, cat, num):
    ret = "\t".join([cat, str(num), '', line])
    return ret

In [47]:
# dir上のzipファイル名を取得
a_zips = glob.glob("%s/*.zip" % akuta_dir)

In [58]:
# 芥川龍之介の著作からある程度ランダムにデータを選んでtsvに出力
min_len = 60
max_len = 240
with open(akuta_tsv, "w") as wf:
    # 全zipを処理
    for fname in a_zips:
        with zipfile.ZipFile(fname, 'r') as f:
            tfile = f.namelist()[0]
            with io.StringIO(f.read(tfile).decode('CP932', 'ignore')) as sf:
                for line in preprocess(sf):
                    if min_len < len(line) < max_len: # 長さ制限
                        tline = to_tsv(line, "akuta", 0)
                        wf.write(tline)

In [59]:
# 宮沢健二の著作からある程度ランダムにデータを選んでtsvに出力
m_zips = glob.glob("%s/*.zip" % miya_dir)
min_len = 60
max_len = 240
with open(miya_tsv, "w") as wf:
    # 全zipを処理
    for fname in a_zips:
        with zipfile.ZipFile(fname, 'r') as f:
            tfile = f.namelist()[0]
            with io.StringIO(f.read(tfile).decode('CP932', 'ignore')) as sf:
                for line in preprocess(sf):
                    if min_len < len(line) < max_len: # 長さ制限
                        tline = to_tsv(line, "miya", 1)
                        wf.write(tline)

In [48]:
# tmpに全部変換しつつ展開
for fname in a_zips:
    with zipfile.ZipFile(fname, 'r') as f:
        tfile = f.namelist()[0]
        out_fname = "tmp/%s" % tfile
        with open(out_fname, 'w') as wf, io.StringIO(f.read(tfile).decode('CP932', "ignore")) as f:
            for line in preprocess(f):
                wf.write(line)

In [18]:
fname = a_zips[0]
with zipfile.ZipFile(fname, 'r') as f:
    #print(f)
    print(f.namelist())
    cont = f.read(f.namelist()[0])

['oshino.txt']


In [33]:
with io.StringIO(cont.decode('CP932')) as f:
    preprocess(f)

　ここは南蛮寺の堂内である。ふだんならばまだ硝子画の窓に日の光の当っている時分であろう。が、今日は梅雨曇りだけに、日の暮の暗さと変りはない。その中にただゴティック風の柱がぼんやり木の肌を光らせながら、高だかとレクトリウムを守っている。それからずっと堂の奥に常燈明の油火が一つ、龕の中に佇んだ聖者の像を照らしている。参詣人はもう一人もいない。

　そう云う薄暗い堂内に紅毛人の神父が一人、祈祷の頭を垂れている。年は四十五六であろう。額の狭い、顴骨の突き出た、頬鬚の深い男である。床の上に引きずった着物は「あびと」と称える僧衣らしい。そう云えば「こんたつ」と称える念珠も手頸を一巻き巻いた後、かすかに青珠を垂らしている。

　堂内は勿論ひっそりしている。神父はいつまでも身動きをしない。

　そこへ日本人の女が一人、静かに堂内へはいって来た。紋を染めた古帷子に何か黒い帯をしめた、武家の女房らしい女である。これはまだ三十代であろう。が、ちょいと見たところは年よりはずっとふけて見える。第一妙に顔色が悪い。目のまわりも黒い暈をとっている。しかし大体の目鼻だちは美しいと言っても差支えない。いや、端正に過ぎる結果、むしろ険のあるくらいである。

　女はさも珍らしそうに聖水盤や祈祷机を見ながら、怯ず怯ず堂の奥へ歩み寄った。すると薄暗い聖壇の前に神父が一人いている。女はやや驚いたように、ぴたりとそこへ足を止めた。が、相手の祈祷していることは直にそれと察せられたらしい。女は神父を眺めたまま、黙然とそこに佇んでいる。

　堂内は不相変ひっそりしている。神父も身動きをしなければ、女も眉一つ動かさない。それがかなり長い間であった。

　その内に神父は祈祷をやめると、やっと床から身を起した。見れば前には女が一人、何か云いたげに佇んでいる。南蛮寺の堂内へはただ見慣れぬ磔仏を見物に来るものも稀ではない。しかしこの女のここへ来たのは物好きだけではなさそうである。神父はわざと微笑しながら、片言に近い日本語を使った。

「何か御用ですか？」

「はい、少々お願いの筋がございまして。」

　女は慇懃に会釈をした。貧しい身なりにも関らず、これだけはちゃんと結い上げた笄髷の頭を下げたのである。神父は微笑んだ眼に目礼した。手は青珠の「こんたつ」に指をからめたり離したりしている。

「わたくしは一番ヶ半兵衛の後家、しの